# Let's Implement LeNet Architecture from scratch using PyTorch Library

Importing the Libraries

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch import optim
import torch.nn.functional as F

# Main Focus:: Building LeNet Model

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.relu= nn.ReLU()
        self.pool= nn.AvgPool2d(kernel_size= (2, 2), stride=(2, 2))
        self.conv1=nn.Conv2d(in_channels=1, out_channels= 6, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.conv2=nn.Conv2d(in_channels=6, out_channels= 16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.conv3=nn.Conv2d(in_channels=16, out_channels= 120, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.linear1=nn.Linear(120, 84)
        self.linear2=nn.Linear(84, 10)
    
    def forward(self, x):
        x= self.relu(self.conv1(x))
        x= self.pool(x)
        x= self.relu(self.conv2(x))
        x= self.pool(x)
        x= self.relu(self.conv3(x))
        x= x.reshape(x.shape[0], -1)
        x= self.relu(self.linear1(x))
        x= self.linear2(x)
        return x

# Adding Device

In [3]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameter

In [4]:
in_channels= 1
num_classes= 10
learning_rate= 0.1
batch_size= 64
num_epochs= 3

# Since LeNet takes 32X32 input size, but MNIST is just 28X28 size, So Let's work on padding the MNIST Dataset

In [5]:
transform= transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor()
])

# Importing datasets and initializing the dataset

In [6]:
train_dataset= datasets.MNIST(root="dataset/", train=True, transform=transform, download= True)
test_dataset= datasets.MNIST(root="dataset/", train=False, transform= transform, download= True)

train_loader= DataLoader(dataset= train_dataset, batch_size= batch_size, shuffle= True)
test_loader= DataLoader(dataset= test_dataset, batch_size= batch_size, shuffle= True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 10397967.42it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 339735.34it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3192319.20it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1895763.63it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



# Model

In [7]:
model= LeNet().to(device)

# Loss Functions and Activation Functions

In [8]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(), lr= learning_rate)

# Let's Train the network

In [9]:
#train network

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data= data.to(device= device)
        targets= targets.to(device= device)
        
        scores= model(data)
        loss= criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
    

100%|██████████| 938/938 [00:22<00:00, 41.63it/s]


# Checking the Accuracy 

In [10]:
#Check accuracy on training & Test to see how good our model

def check_accuracy(loader, model):
    num_correct= 0
    num_samples= 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x= x.to(device= device)
            y= y.to(device= device)
            
            scores= model(x)
            _, predictions= scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
    model.train()
    return num_correct/num_samples

# Printing the Accuracy

In [11]:
print(f" Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f" Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

 Accuracy on training set: 9.74
 Accuracy on test set: 9.82
